In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

In [ ]:
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)                     
    return df

In [2]:
df=pd.read_csv('train.csv')
x_test=pd.read_csv('test.csv')

ids=x_test['PassengerId']

df=df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)
x_test=x_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode())

x_test['Age'] = x_test['Age'].fillna(x_test['Age'].median())
x_test['Fare'] = x_test['Fare'].fillna(x_test['Fare'].mean())

df = pd.get_dummies(df, columns=['Sex','Embarked'], prefix=['Sex','Embarked'],dtype=int)
x_test = pd.get_dummies(x_test, columns=['Sex','Embarked'], prefix=['Sex','Embarked'],dtype=int)


y_train=df['Survived']
df=df.drop(['Survived'],axis=1)

In [3]:
df.isna().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [4]:
x_test.isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [5]:


#scaling the data using standard scaler
'''scaler=StandardScaler()
x_train=scaler.fit_transform(x_train[['Age','SibSp','Parch','Fare']])
x_test=scaler.transform(x_test[['Age','SibSp','Parch','Fare']])
x_cv=scaler.transform(x_cv[['Age','SibSp','Parch','Fare']])'''




"scaler=StandardScaler()\nx_train=scaler.fit_transform(x_train[['Age','SibSp','Parch','Fare']])\nx_test=scaler.transform(x_test[['Age','SibSp','Parch','Fare']])\nx_cv=scaler.transform(x_cv[['Age','SibSp','Parch','Fare']])"

In [6]:
#split the data
x_train, x_cv, y_train, y_cv = train_test_split(df, y_train,test_size=0.20,random_state=32)

# Create polynomial features (degree 2)
poly = PolynomialFeatures(2, include_bias=False)
x_train=poly.fit_transform(x_train)
x_cv=poly.transform(x_cv)
x_test=poly.transform(x_test)

In [7]:
model=LogisticRegression(max_iter=100)
model.fit(x_train,y_train)
print(model.score(x_train,y_train))
print(model.score(x_cv,y_cv))

0.8089887640449438
0.8044692737430168


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
y_pred=model.predict(x_test)
ids=list(ids)
submit=pd.DataFrame({'PassengerId':ids,'Survived':y_pred})

In [16]:
print(submit)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [17]:
submit2.to_csv('submit.csv',index=False)

In [18]:
s=pd.read_csv('submit.csv')

In [19]:
s

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
